# Import

In [ ]:
import numpy as np
import pandas as pd
import json
from konlpy.tag import Kkma, Twitter
import re
import gensim

In [2]:
pokr = pd.read_json('./pokr.json')

In [3]:
pokr[:1]

,content,date,id,meeting_id,meeting_title,person_id,sequence,speaker,url
0,의석을 정돈해 주시기 바랍니다. \n성원이 되었으므로 제4차 본회의를 개의하겠습니...,2012-07-17,1077715,19309444267,본회의,19461111.0,0,의장 강창희,http://pokr.kr/meeting/19309444267/dialog


In [4]:
pokr = pokr[['content']] # 발언들만 사용

In [5]:
Tw = Twitter()

# Word2Vec에는 전체 형태소, LDA에는 명사만 사용

In [6]:
Tw.morphs('성원이 되었으므로 제 342회 회의를')

['성원', '이', '되었', '으므로', '제', '342', '회', '회의', '를']

In [7]:
Tw.nouns('성원이 되었으므로 제 342회 회의를')

['성원', '제', '회', '회의']

In [52]:
word2vec_m = []
lda_n = []
for row in pokr.iterrows():
    content = row[1]['content']
    
    real_content = (re.sub('[=……ㆍ◎,◯!―()\n]', '', content))
    
    morphs = Tw.morphs(real_content)
    nouns = Tw.nouns(real_content)
    
    word2vec_m.append(morphs)
    lda_n.append(nouns)

In [ ]:
dictionary = corpora.Dictionary(lda_n)

In [62]:
corpus = [dictionary.doc2bow(comment) for comment in lda_n]

In [63]:
word2vecmodel = gensim.models.Word2Vec(all_comment,size=200,min_count=10, sg=1)
ldamodel = gensim.models.ldamodel.LdaModel(corpus = corpus, num_topics = 30, id2word = dictionary, update_every=1, passes=5)

# 결과 확인

In [9]:
word2vecmodel.most_similar(positive=['새누리당', '야당'],negative=['민주당'])

[('여당', 0.845704972743988),
 ('여야', 0.5596292018890381),
 ('조원진', 0.5339096188545227),
 ('자당', 0.5309696197509766),
 ('당인', 0.5138304829597473),
 ('환노', 0.513545036315918),
 ('황우여', 0.5115187764167786),
 ('정개', 0.5002454519271851),
 ('윤상현', 0.49903807044029236),
 ('김기현', 0.49822482466697693)]

In [10]:
ldamodel.show_topic(1)

[('제도', 0.055089469180403257),
 ('재정', 0.034219618045418614),
 ('보험', 0.029871074229278288),
 ('공무원', 0.02635959738100898),
 ('복지', 0.025001340775675784),
 ('사회', 0.024382146782972854),
 ('개선', 0.021972295858315494),
 ('여성', 0.020580552381803108),
 ('부담', 0.016235734811720894),
 ('연금', 0.013923276650965573)]

In [ ]:
word2vecmodel.save('word2vecmodel')
ldamodel.save('ldamodel')